In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

In [42]:
stfold = StratifiedKFold(n_splits=5, shuffle=True)
train_l = list()
val_l = list()
test_l = list()
maxlen = 100
cnt = 0
for train_index, test_index in stfold.split(data['comment'], data['label']):
    cnt += 1
    train = data.iloc[train_index]
    train_x, train_y = train['comment'], train['label']
    tmp = data.iloc[test_index]

    val, test = train_test_split(tmp, test_size=0.5, stratify=tmp['label'])    
    val_x, val_y = val['comment'], val['label']
    test_x, test_y = test['comment'], test['label']    
    
    train_x = keras.preprocessing.sequence.pad_sequences(train_x, maxlen=maxlen)
    val_x = keras.preprocessing.sequence.pad_sequences(val_x, maxlen=maxlen)
    test_x = keras.preprocessing.sequence.pad_sequences(test_x, maxlen=maxlen)
    
    embed_dim = 128  # Embedding size for each token
    vocab_size = n_word_unique + 1

    inputs = layers.Input(shape=(maxlen,))
    embedding_layer = layers.Embedding(input_dim = vocab_size, output_dim = embed_dim, input_length=maxlen)
    x = embedding_layer(inputs)
    x = layers.Bidirectional(layers.LSTM(128, input_dim=(None, embed_dim), 
                                         return_sequences=False, activation='tanh', 
                                         dropout=0.4))(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)

    model = keras.Model(inputs=inputs, outputs=outputs)
    model_path = "lstm_models/{itr:04}/".format(itr=cnt)
    checkpoint_path = model_path + "{epoch:04d}.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)
    !mkdir $model_path
    
    model.compile(tf.keras.optimizers.Adam(lr=0.001), "binary_crossentropy", metrics=["accuracy", 
                                                          tf.keras.metrics.AUC(name='auc')])
    
    
    model.save_weights(checkpoint_path.format(epoch=0))

    checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_auc', verbose=1,
            save_weights_only=True, save_best_only=True, mode='max')
    
    history = model.fit(
                        train_x, train_y, 
                        batch_size=128, epochs=10, 
                        validation_data=(val_x, val_y),
                        callbacks = [checkpoint]
                        )
    
    latest = tf.train.latest_checkpoint(checkpoint_dir)
    model.load_weights(latest)
    
    train_eval = model.evaluate(train_x, train_y)
    val_eval = model.evaluate(val_x, val_y)
    test_eval = model.evaluate(test_x, test_y)
    
    test_l.append(test_eval)
    val_l.append(val_eval)
    train_l.append(train_eval)

mkdir: cannot create directory ‘lstm_models/0001/’: No such file or directory
Epoch 1/10
63/63 [==============================] - ETA: 0s - loss: 0.6588 - accuracy: 0.5928 - auc: 0.6401
Epoch 00001: val_auc improved from -inf to 0.74992, saving model to lstm_models/0001/0001.ckpt
63/63 [==============================] - 25s 389ms/step - loss: 0.6588 - accuracy: 0.5928 - auc: 0.6401 - val_loss: 0.6064 - val_accuracy: 0.6730 - val_auc: 0.7499
Epoch 2/10
63/63 [==============================] - ETA: 0s - loss: 0.3411 - accuracy: 0.8689 - auc: 0.9329
Epoch 00002: val_auc improved from 0.74992 to 0.90623, saving model to lstm_models/0001/0002.ckpt
63/63 [==============================] - 28s 446ms/step - loss: 0.3411 - accuracy: 0.8689 - auc: 0.9329 - val_loss: 0.3777 - val_accuracy: 0.8540 - val_auc: 0.9062
Epoch 3/10
63/63 [==============================] - ETA: 0s - loss: 0.0887 - accuracy: 0.9697 - auc: 0.9950
Epoch 00003: val_auc improved from 0.90623 to 0.91027, saving model to lstm_m

In [43]:
test_l = np.array(test_l)
val_l = np.array(val_l)
train_l = np.array(train_l)

print("test avg loss: ", np.mean(test_l[:, 0]), "+/-" ,np.std(test_l[:, 0]))
print("test avg acc: ", np.mean(test_l[:, 1]),  "+/-" ,np.std(test_l[:, 1]))
print("test avg auc: ", np.mean(test_l[:, 2]),  "+/-" ,np.std(test_l[:, 2]))

print('\n')


print("val avg loss: ", np.mean(val_l[:, 0]), "+/-" ,np.std(val_l[:, 0]))
print("val avg acc: ", np.mean(val_l[:, 1]),  "+/-" ,np.std(val_l[:, 1]))
print("val avg auc: ", np.mean(val_l[:, 2]),  "+/-" ,np.std(val_l[:, 2]))

print('\n')


print("train avg loss: ", np.mean(train_l[:, 0]), "+/-" ,np.std(train_l[:, 0]))
print("train avg acc: ", np.mean(train_l[:, 1]),  "+/-" ,np.std(train_l[:, 1]))
print("train avg auc: ", np.mean(train_l[:, 2]),  "+/-" ,np.std(train_l[:, 2]))


test avg loss:  0.42503719329833983 +/- 0.057295440283045514
test avg acc:  0.8563999891281128 +/- 0.005953160688657734
test avg auc:  0.9191849112510682 +/- 0.007085450751051029


val avg loss:  0.4581384897232056 +/- 0.052439362762692635
val avg acc:  0.846999990940094 +/- 0.010862772326479241
val avg auc:  0.9091739416122436 +/- 0.007798309534021888


train avg loss:  0.01582770240493119 +/- 0.011341890797786082
train avg acc:  0.997249984741211 +/- 0.002419960481045956
train avg auc:  0.9996478199958801 +/- 0.0003461450677962361
